# Meridional Heat Transport (MHT)
This notebook calculates the model MHT from two ways, based on distinct MOM5 diagnostics:

1. `temp_yflux_adv_int_z` (depth-integrated meridional heat transport due to resovled advection) and

2. `net_sfc_heating` (net surface heat flux).

To use this notebook, we need to ensure this diagnostic is available in the model output. We can check if a variable is available using `cc.querying.get_variables()` function from the COSIMA cookbook.  
  
Note that currently this notebook calculates the total (all basins) MHT and it also includes comparisons to a few observational products.

## Loading relevant libraries

In [1]:
import intake
catalog = intake.cat.access_nri
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
%matplotlib inline

## Loading model data

We will use the COSIMA cookbook to search its database and load `temp_yflux_adv_int_z` to start our calculations. To do this we will need to do the following:
1. Start a COSIMA cookbook session
2. Define the model configuration of interest
3. Define the experiment of interest
4. Define the diagnostic (variable) of interest

**NOTE:** If you are in doubt about the models, experiments and diagnostics available in the database, check the [Cookbook Tutorial](../Tutorials/COSIMA_CookBook_Tutorial.ipynb) for more information.

In [2]:
from dask.distributed import Client
client = Client("tcp://10.6.62.1:8786")
client

<Client: 'tcp://10.6.62.1:8786' processes=1 threads=12, memory=46.00 GiB>

In [3]:
#Start a COSIMA cookbook session

#Define experiment of interest
experiment = '025deg_jra55_iaf_omip2_cycle2'

We are now ready to query the database and load the data to start our analysis. We load `temp_yflux_adv_int_z`. For this example, we have chosen to use 6 years of output.

## Method 1: using depth-integrated meridional heat transport due to resovled advection

In [4]:
cat_subset = catalog[experiment]
var_search = cat_subset.search(variable='temp_yflux_adv_int_z')
darray = var_search.to_dask()
darray = darray['temp_yflux_adv_int_z']
mht_method1 = darray
mht_method1

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)


<xarray.DataArray 'temp_yflux_adv_int_z' (time: 732, yu_ocean: 1080,
                                          xt_ocean: 1440)>
dask.array<concatenate, shape=(732, 1080, 1440), dtype=float32, chunksize=(1, 216, 240), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -279.9 -279.6 -279.4 ... 79.38 79.62 79.88
  * time      (time) datetime64[ns] 1958-01-14T12:00:00 ... 2018-12-14T12:00:00
  * yu_ocean  (yu_ocean) float64 -81.02 -80.92 -80.81 ... 89.79 89.89 90.0
Attributes:
    long_name:      z-integral of cp*rho*dxt*v*temp
    units:          Watts
    valid_range:    [-1.e+18  1.e+18]
    cell_methods:   time: mean
    time_avg_info:  average_T1,average_T2,average_DT

Viewing the variable containing our data provides a number of key information, including:
- *Shape of dataset* - In this case the data includes 72 time steps, 1080 steps along the y axis and 1440 along the x axis.
- *Name of coordinates* - Our data includes `time`, `yu_ocean` (i.e., latitude), and `xt_ocean` (i.e., longitude).
- *Values included under each coordinate* - We can see that `time` includes monthly values from 2198-01-14 to 2203-12-14.
- *Metadata* - This information is included under `Attributes` and it includes things like units.

## Calculating mean and converting units
From the metadata, we can see that our dataset is in Watts (W), so we will convert them to petawatts (PW)

In [5]:
#We transform units from watts to petawatts
mht_method1 = mht_method1 * 1e-15
mht_method1 = mht_method1.assign_attrs(units='PettaWatts')
mht_method1

<xarray.DataArray 'temp_yflux_adv_int_z' (time: 732, yu_ocean: 1080,
                                          xt_ocean: 1440)>
dask.array<mul, shape=(732, 1080, 1440), dtype=float32, chunksize=(1, 216, 240), chunktype=numpy.ndarray>
Coordinates:
  * xt_ocean  (xt_ocean) float64 -279.9 -279.6 -279.4 ... 79.38 79.62 79.88
  * time      (time) datetime64[ns] 1958-01-14T12:00:00 ... 2018-12-14T12:00:00
  * yu_ocean  (yu_ocean) float64 -81.02 -80.92 -80.81 ... 89.79 89.89 90.0
Attributes:
    units:    PettaWatts

and then we compute the mean across `time` and sum over all longitudes.

In [6]:
mht_method1 = mht_method1.mean('time').sum('xt_ocean')
mht_method1

<xarray.DataArray 'temp_yflux_adv_int_z' (yu_ocean: 1080)>
dask.array<sum-aggregate, shape=(1080,), dtype=float32, chunksize=(216,), chunktype=numpy.ndarray>
Coordinates:
  * yu_ocean  (yu_ocean) float64 -81.02 -80.92 -80.81 ... 89.79 89.89 90.0

## Method 2: using from the net surface heat flux (assuming steady state)

First, we load the surface heat flux and grid metrics:

In [11]:
cat_subset = catalog[experiment]
# var_search = cat_subset.search(variable='yt_ocean')
#var_search = var_search.search(path=var_search.df['path'][0])
sorted(set().union(*cat_subset.df['variable']))

['ANGLE',
 'ANGLET',
 'HTE',
 'HTN',
 'NCAT',
 'TLAT',
 'TLON',
 'Tair_m',
 'Tsfc_m',
 'ULAT',
 'ULON',
 'age_global',
 'agm',
 'aice',
 'aice_m',
 'aicen_m',
 'aiso_bih',
 'albice_m',
 'albsni_m',
 'albsno_m',
 'alidf_ai_m',
 'alidr_ai_m',
 'alvdf_ai_m',
 'alvdr_ai_m',
 'area_t',
 'area_u',
 'aredi',
 'average_DT',
 'average_T1',
 'average_T2',
 'bih_fric_u',
 'bih_fric_v',
 'blkmask',
 'bmf_u',
 'bmf_v',
 'bottom_salt',
 'bottom_temp',
 'bottom_temp_max',
 'buoyfreq2_wt',
 'bv_freq',
 'congel',
 'congel_m',
 'daidtd',
 'daidtd_m',
 'daidtt',
 'daidtt_m',
 'dht',
 'diff_cbt_s',
 'diff_cbt_t',
 'divu_m',
 'drag_coeff',
 'dvidtd',
 'dvidtd_m',
 'dvidtt',
 'dvidtt_m',
 'dvirdgdt_m',
 'dxt',
 'dxu',
 'dyt',
 'dyu',
 'ekman_we',
 'eta_adjust',
 'eta_global',
 'eta_nonbouss',
 'eta_t',
 'evap',
 'evap_ai_m',
 'evap_heat',
 'fcondtop_ai',
 'fcondtop_ai_m',
 'fcondtopn_ai_m',
 'fhocn_ai_m',
 'flat_ai_m',
 'flatn_ai_m',
 'flwdn_m',
 'flwup_ai_m',
 'fmeltt_ai_m',
 'fmelttn_ai_m',
 'fprec',
 'fp

In [7]:
cat_subset = catalog[experiment]
var_search = cat_subset.search(variable='net_sfc_heating')
darray = var_search.to_dask()
darray = darray['net_sfc_heating']
shflux = darray
shflux_am = shflux.mean('time').load()

cat_subset = catalog[experiment]
var_search = cat_subset.search(variable='area_t')
var_search = var_search.search(path=var_search.df['path'][0])
darray = var_search.to_dask()
darray = darray['area_t']
area = darray
cat_subset = catalog[experiment]
var_search = cat_subset.search(variable='geolat_t')
var_search = var_search.search(path=var_search.df['path'][0])
darray = var_search.to_dask()
darray = darray['geolat_t']
lat  = darray
cat_subset = catalog[experiment]
var_search = cat_subset.search(variable='yt_ocean') # this is the problem!
var_search = var_search.search(path=var_search.df['path'][0])
darray = var_search.to_dask()
darray = darray['yt_ocean']
latv = darray

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use D

KeyError: 0

Now calculate Meridional Heat Flux (MHF):

$\textrm{MHF} = \textrm{Cummulative sum of } (\textrm{SHFLUX} \times \textrm{AREA}) \textrm{ along latitudes}$

**Note**: The following cell might take 1-2 min.

In [ ]:
%%time

mhf = xr.zeros_like(latv)

for i in range(len(latv)):
    inds = lat < latv[i]
    atmp = area.where(lat < latv[i])
    stmp = shflux_am.where(lat < latv[i])
    mhf[i] = np.sum(atmp * stmp)

mht_method2 = mhf + (mhf[0] - mhf[-1])/2

In [ ]:
#We transform units from watts to petawatts
mht_method2 = mht_method2 * 1e-15
mht_method2 = mht_method2.assign_attrs(units='PettaWatts')
mht_method2

## Comparison between model output and observations
Before producing our figure, we will compare the model output with observations to check the model accuracy.

### Read ERBE Period Ocean and Atmospheric Heat Transport
Observations are also available in Gadi, here we show how to load them to our notebook.

In [ ]:
#Path to the file containing observations
filename = '/g/data3/hh5/tmp/cosima/observations/original/MHT/obs_vq_am_estimates.txt'

#Creating empty variables to store our observations
erbe_mht = []
erbe_lat = []

#Opening data and saving it to empty variables above
with open(filename) as f:
    #Open each line from rows 1 to 96
    for line in f.readlines()[1:96]:
        #Separating each line to extract data
        line = line.strip()
        sline = line.split()
        #Extracting latitude and MHT and saving to empty variables
        erbe_lat.append(float(sline[0]))
        erbe_mht.append(float(sline[3]))

#Checking MHT variables
erbe_mht

### Read NCEP and ECMWF Oceanic and Atmospheric Transport Products

In [ ]:
#Path to the file containing observations
filename = '/g/data3/hh5/tmp/cosima/observations/original/MHT/ANNUAL_TRANSPORTS_1985_1989.ascii'

#Creating empty variables to store our observations
ncep_g_mht = []
ecwmf_g_mht = []
ncep_g_err = []
ecwmf_g_err = []
ncep_a_mht = []
ecwmf_a_mht = []
ncep_a_err = []
ecwmf_a_err = []
ncep_p_mht = []
ecwmf_p_mht = []
ncep_p_err = []
ecwmf_p_err = []
ncep_i_mht = []
ecwmf_i_mht = []
ncep_i_err = []
ecwmf_i_err = []
ncep_ip_mht = []
ecwmf_ip_mht = []
ncep_ip_err = []
ecwmf_ip_err = []
o_lat = []

#Opening data and saving it to empty variables above
with open(filename) as f:
#Open each line in file (ignoring the first row)
    for line in f.readlines()[1:]:
        #Separating each line to extract data
        line = line.strip()
        sline = line.split()
        #Extracting values and saving to correct variable defined above
        o_lat.append(float(sline[0])*0.01)  # T42 latitudes (north to south)
        ncep_g_mht.append(float(sline[4])*0.01) # Residual Ocean Transport - NCEP
        ecwmf_g_mht.append(float(sline[5])*0.01)# Residual Ocean Transport - ECWMF
        ncep_a_mht.append(float(sline[7])*0.01)# Atlantic Ocean Basin Transport - NCEP
        ncep_p_mht.append(float(sline[8])*0.01)# Pacific Ocean Basin Transport - NCEP
        ncep_i_mht.append(float(sline[9])*0.01)# Indian Ocean Basin Transport - NCEP
        ncep_g_err.append(float(sline[10])*0.01)# Error Bars for NCEP Total Transports
        ncep_a_err.append(float(sline[11])*0.01)# Error Bars for NCEP Atlantic Transports 
        ncep_p_err.append(float(sline[12])*0.01)# Error Bars for NCEP Pacific Transports 
        ncep_i_err.append(float(sline[13])*0.01)# Error Bars for NCEP Indian Transports 
        ecwmf_a_mht.append(float(sline[15])*0.01)# Atlantic Ocean Basin Transport - ECWMF
        ecwmf_p_mht.append(float(sline[16])*0.01)# Pacific Ocean Basin Transport - ECWMF
        ecwmf_i_mht.append(float(sline[17])*0.01)# Indian Ocean Basin Transport - ECWMF
        ecwmf_g_err.append(float(sline[18])*0.01)# Error Bars for ECWMF Total Transports
        ecwmf_a_err.append(float(sline[19])*0.01)# Error Bars for NCEP Atlantic Transports
        ecwmf_p_err.append(float(sline[20])*0.01)# Error Bars for NCEP Pacific Transports
        ecwmf_i_err.append(float(sline[21])*0.01)# Error Bars for NCEP Indian Transports

#Calculating MHT
ncep_ip_mht  = [a+b for a, b in zip(ncep_p_mht,ncep_i_mht)]
ecwmf_ip_mht = [a+b for a, b in zip(ecwmf_p_mht,ecwmf_i_mht)]
ncep_ip_err  = [max(a, b) for a, b in zip(ncep_p_err, ncep_i_err)]
ecwmf_ip_err = [max(a, b) for a, b in zip(ecwmf_p_err, ecwmf_i_err)]

## Plotting model outputs against observations

We plot the global meridional heat transport as calculated from model outputs (blue line) and observations.

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax = fig.add_subplot(1, 1, 1)

#Plotting MHT from model outputs
mht_method1.plot(ax = ax, color="blue", label="ACCESS-OM2-025")

mht_method2.plot(ax = ax, color="green", label="ACCESS-OM2-025 (shflux)")

#Adding observations and error bars for observations
ax.plot(erbe_lat, erbe_mht, 'k--', linewidth=1, label="ERBE, JRA-25, NCEP/NCAR and ERA40")
plt.errorbar(o_lat[::-1], ncep_g_mht[::-1], yerr=ncep_g_err[::-1], c='gray', fmt='s', 
             markerfacecolor='k', markersize=3, capsize=2, linewidth=1, label="NCEP")
plt.errorbar(o_lat[::-1], ecwmf_g_mht[::-1], yerr=ecwmf_g_err[::-1], c='gray', fmt='D', 
             markerfacecolor='white', markersize=3, capsize=2, linewidth=1, label="ECWMF")
plt.errorbar(24, 1.5, yerr=0.3, fmt='o', c='black', markersize=3, capsize=2, linewidth=1, 
             label="Macdonald and Wunsch 1996")
plt.errorbar(-30, -0.9, yerr=0.3, fmt='o', c='black', markersize=3, capsize=2, linewidth=1)
plt.errorbar(24, 2.0, yerr=0.3, fmt='x', c='green', markersize=3, capsize=2, linewidth=1, 
             label="Lavin et al. and Bryden et al.")
plt.errorbar(24, 1.83, yerr=0.28, fmt='^', c='red', markersize=4, capsize=2, linewidth=1, 
             label="Ganachaud and Wunsch 2003")
plt.errorbar(-30, -0.6, yerr=0.3, fmt='^', c='red', markersize=4, capsize=2, linewidth=1)
plt.errorbar(-19, -0.8, yerr=0.3, fmt='^', c='red', markersize=4, capsize=2, linewidth=1)
plt.errorbar(47, 0.6, yerr=0.1, fmt='^', c='red', markersize=4, capsize=2, linewidth=1)

#Adding legend
plt.legend(frameon=False, fontsize=12)
plt.axhline(y=0, linewidth=1, color='black')

#Defining plot limits along the y axis
plt.ylim(-2.25, 2.5)

#Adding titles for figure and axes
plt.title('Global Ocean Meridional Heat Transport', fontsize=18)
plt.xlabel('Latitude')
plt.ylabel('$10^{15}$ Watts');